In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats as ss
import itertools 
from collections import Counter
from dython.nominal import associations

%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Naše dáta obsahujú viac ako 100 MB záznamov, mnohé kategorické premenné a pod. a preto si potrebujeme najprv upraviť dataset, predpripraviť ho na použitie a následne importovat do našej navrdhnutej siete.

Ako výsledok dosiahneme očistené transformované dáta od outlierov, nepresných hodnôt a použijeme subset dát na základe kritérií, ktorý si následne vzesportujeme do csv pre daľšie použitie.

Aby sme mohli pracovat s datami, spravit korelaciu a podobne potrebujeme prerobit cely dataset na numericke hodnoty

Nase data budu obsahovat, ze

END result will get 2 files: one csv, and other npy encoded

In [4]:
# načítanie povodného súboru do premennje file sktorou budeme pracovat
file = pd.read_csv("database.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
# skontrolujeme aký tvar má databáza, s ktorou budeme pracovať
file.shape

(638454, 24)

In [6]:
# vidíme, že Peretrator Age zaradilo ako objekt napriek tomu, že to má byť int. 
# Indikuje chýbajúce hodonoty alebo hodoty nie int 
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638454 entries, 0 to 638453
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   Record ID              638454 non-null  int64 
 1   Agency Code            638454 non-null  object
 2   Agency Name            638454 non-null  object
 3   Agency Type            638454 non-null  object
 4   City                   638454 non-null  object
 5   State                  638454 non-null  object
 6   Year                   638454 non-null  int64 
 7   Month                  638454 non-null  object
 8   Incident               638454 non-null  int64 
 9   Crime Type             638454 non-null  object
 10  Crime Solved           638454 non-null  object
 11  Victim Sex             638454 non-null  object
 12  Victim Age             638454 non-null  int64 
 13  Victim Race            638454 non-null  object
 14  Victim Ethnicity       638454 non-null  object
 15  

In [7]:
# Premenima na numerical s tým, že nenumerická hodnota sa nahradí NaM
file['Perpetrator Age']=pd.to_numeric(file['Perpetrator Age'], errors='coerce')

In [8]:
# vidime, ze mame jened NaN, ten riadok mozeme dropnut
file.isnull().sum()

Record ID                0
Agency Code              0
Agency Name              0
Agency Type              0
City                     0
State                    0
Year                     0
Month                    0
Incident                 0
Crime Type               0
Crime Solved             0
Victim Sex               0
Victim Age               0
Victim Race              0
Victim Ethnicity         0
Perpetrator Sex          0
Perpetrator Age          1
Perpetrator Race         0
Perpetrator Ethnicity    0
Relationship             0
Weapon                   0
Victim Count             0
Perpetrator Count        0
Record Source            0
dtype: int64

In [9]:
file = file.dropna()

In [12]:
file.describe()

,Record ID,Year,Incident,Victim Age,Perpetrator Age,Victim Count,Perpetrator Count
count,638453.000000,638453.000000,638453.000000,638453.000000,638453.000000,638453.000000,638453.000000
mean,319227.005932,1995.801074,22.967797,35.033495,20.322697,0.123334,0.185224
std,184305.660258,9.927675,92.149837,41.628336,17.886842,0.537734,0.585496
min,1.000000,1980.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,159614.000000,1987.000000,1.000000,22.000000,0.000000,0.000000,0.000000
50%,319227.000000,1995.000000,2.000000,30.000000,21.000000,0.000000,0.000000
75%,478840.000000,2004.000000,10.000000,42.000000,31.000000,0.000000,0.000000
max,638454.000000,2014.000000,999.000000,998.000000,99.000000,10.000000,10.000000


In [13]:
# Victime age Max je 998, co je divné čislo, preto odstránime všetkz riadky co majú 998 ako hodnotu
file[file['Victim Age'] == 998].shape[0]

974

In [14]:
temp_to_delete = []
for x,y in zip(file['Record ID'],file['Victim Age']):
    if y==998:
        temp_to_delete.append(x-1)
    

In [15]:
file = file.drop(labels=temp_to_delete, axis=0)

In [16]:
file[file['Victim Age'] == 998].shape[0]

0

In [17]:
# Uz to vyzera lepse ale stale nedostacujuce
file.describe()

,Record ID,Year,Incident,Victim Age,Perpetrator Age,Victim Count,Perpetrator Count
count,637479.000000,637479.000000,637479.000000,637479.000000,637479.000000,637479.000000,637479.000000
mean,319114.355631,1995.795041,22.987975,33.562185,20.338024,0.123303,0.185308
std,184278.864201,9.925972,92.213602,17.792601,17.885289,0.537756,0.585502
min,1.000000,1980.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,159530.500000,1987.000000,1.000000,22.000000,0.000000,0.000000,0.000000
50%,319054.000000,1995.000000,2.000000,30.000000,21.000000,0.000000,0.000000
75%,478706.500000,2004.000000,10.000000,42.000000,31.000000,0.000000,0.000000
max,638454.000000,2014.000000,999.000000,99.000000,99.000000,10.000000,10.000000


In [20]:
# Incident Max je 999, co je divné čislo, preto odstránime všetky riadky co majú 99 ako hodnotu
file[file['Incident'] == 999].shape[0]

16

In [26]:
temp_to_delete.clear()
for x,y in zip(file['Record ID'],file['Incident']):
    if y==999:
        temp_to_delete.append(x-1)
    

In [27]:
temp_to_delete

[420921,
 437186,
 453603,
 470022,
 486417,
 503368,
 520240,
 521240,
 537536,
 552928,
 568295,
 583061,
 594711,
 595711,
 599658,
 627946]

In [29]:
print(file.loc[[420921]])

        Record ID Agency Code Agency Name    Agency Type  City    State  Year  \
420921     420922     FL01300  Miami-Dade  County Police  Dade  Florida  2001   

        Month  Incident              Crime Type  ... Victim Ethnicity  \
420921  April       999  Murder or Manslaughter  ...          Unknown   

       Perpetrator Sex  Perpetrator Age Perpetrator Race  \
420921         Unknown              0.0          Unknown   

       Perpetrator Ethnicity Relationship   Weapon Victim Count  \
420921               Unknown      Unknown  Handgun            0   

       Perpetrator Count Record Source  
420921                 0          FOIA  

[1 rows x 24 columns]


In [34]:
previous_city = ""
previous_year = 0
previous_month = ''
previous_incident =0
new_file = []
for record, city, year, month, incident in zip(file['Record ID'],file['City'], file['Year'], file['Month'], file['Incident']):
    if (city == previous_city) and (year == previous_year) and (month == previous_month):
            previous_city = city
            previous_year = year
            previous_month = month
            continue
    else:
        previous_city = city
        previous_year = year
        previous_month = month
        new_file.append(record-1)

In [35]:
new_file

[0,
 1,
 3,
 5,
 7,
 10,
 13,
 16,
 17,
 18,
 19,
 20,
 22,
 23,
 24,
 25,
 26,
 29,
 30,
 31,
 32,
 35,
 37,
 41,
 45,
 47,
 48,
 49,
 50,
 51,
 52,
 55,
 58,
 59,
 60,
 61,
 62,
 63,
 65,
 67,
 69,
 71,
 73,
 84,
 91,
 98,
 103,
 108,
 115,
 124,
 135,
 138,
 149,
 156,
 166,
 167,
 168,
 170,
 172,
 173,
 175,
 176,
 177,
 180,
 181,
 184,
 185,
 188,
 191,
 197,
 200,
 205,
 209,
 215,
 221,
 224,
 226,
 232,
 239,
 243,
 246,
 247,
 249,
 252,
 253,
 255,
 256,
 258,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 272,
 275,
 279,
 282,
 285,
 287,
 291,
 292,
 294,
 300,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 330,
 331,
 333,
 335,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 345,
 349,
 350,
 351,
 352,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 371,
 372,
 374,
 375,
 376,
 377,
 379,
 381,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 393,
 

In [36]:
max_crimes_in_period_of_time = file[file.index.isin(new_file)]

In [41]:
max_crimes_in_period_of_time.head(25)

,Record ID,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,...,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count,Record Source
0,1,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,January,1,Murder or Manslaughter,...,Unknown,Male,15.0,Native American/Alaska Native,Unknown,Acquaintance,Blunt Object,0,0,FBI
1,2,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,1,Murder or Manslaughter,...,Unknown,Male,42.0,White,Unknown,Acquaintance,Strangulation,0,0,FBI
3,4,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,1,Murder or Manslaughter,...,Unknown,Male,42.0,White,Unknown,Acquaintance,Strangulation,0,0,FBI
5,6,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,May,1,Murder or Manslaughter,...,Unknown,Male,36.0,White,Unknown,Acquaintance,Rifle,0,0,FBI
7,8,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,June,1,Murder or Manslaughter,...,Unknown,Male,35.0,White,Unknown,Wife,Knife,0,0,FBI
10,11,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,July,1,Murder or Manslaughter,...,Unknown,Unknown,0.0,Unknown,Unknown,Unknown,Unknown,0,1,FBI
13,14,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,August,1,Murder or Manslaughter,...,Unknown,Male,49.0,White,Unknown,Unknown,Fall,0,0,FBI
16,17,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,December,1,Murder or Manslaughter,...,Unknown,Male,19.0,Unknown,Unknown,Acquaintance,Knife,0,0,FBI
17,18,AK00103,Juneau,Municipal Police,Juneau,Alaska,1980,November,1,Murder or Manslaughter,...,Unknown,Male,23.0,Native American/Alaska Native,Unknown,Brother,Blunt Object,0,1,FBI
18,19,AK00106,Nome,Municipal Police,Nome,Alaska,1980,June,1,Murder or Manslaughter,...,Unknown,Male,33.0,Native American/Alaska Native,Unknown,Acquaintance,Handgun,0,0,FBI


In [43]:
max_crimes_in_period_of_time[max_crimes_in_period_of_time['Incident'] == 1].shape[0]

249989

In [42]:
file[file['Incident'] == 1].shape[0]

265034

In [44]:
file['Month'].value_counts()

July         58595
August       57990
December     55086
September    54045
June         53579
October      53561
May          53326
January      52863
March        51366
April        51123
November     49923
February     46022
Name: Month, dtype: int64

In [45]:
max_crimes_in_period_of_time['Month'].value_counts()

July         23828
August       23679
December     22915
October      22637
September    22592
June         22508
May          22439
January      22349
March        22039
April        21874
November     21631
February     20377
Name: Month, dtype: int64